In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import glob
from import_casa import casa

In [3]:
import pickle

In [4]:
with (casa.get_data_path()/"threads/cht-2020-merged.pkl").open("rb") as fin:
    threads = pickle.load(fin)


In [5]:
list(threads[100].opinion_texts())

['5G競標等於是台灣政府做莊坑殺台灣民眾 (第5頁)\u3000該不會是業者出來代風向球吧給你看看業者自己說的話喔5G標金破800億 中華電謝繼茂：千億價格值得！請不要搗亂市場供需原則真懷疑你真是否消費哲']

In [6]:
t0 = threads[107]
t0.print_thread()

Main: <Opinion [mobile01] 威廉BBQ: 在晚上9點多才擠進市政府要打給同事打不通...>
-- <Opinion [mobile01] nicholas820: 這理所當然吧電信業龍頭欸!!!要不然這幾...>
-- <Opinion [mobile01] wingth: 中華很給力說,新年快樂~~~>
-- <Opinion [mobile01] 阿嘍嘍: 中華一直很強 也很貴>
-- <Opinion [mobile01] max-tw: 基地台後面接的固網降價升速每家電信都不塞...>
-- <Opinion [mobile01] YOLOS: 本想稱讚一下中華的但看了樓主的發文紀錄覺...>
-- <Opinion [mobile01] AL921er: 200M應該是10點多中華到巔峰時刻我猜...>
-- <Opinion [mobile01] HermesParis: 哇塞!中華基地台車很強?既然電磁波這麼強...>
-- <Opinion [mobile01] lzx: 刁民們抗議的是沒分到錢，沒錢當然會頭暈想...>
-- <Opinion [mobile01] Tiengap: 覺得都差不多，昨天跨年也跟朋友在101附...>


In [7]:
from io import StringIO
from textwrap import indent, wrap
from datetime import datetime
def format_text(text, textwidth=60, __indent=2, __indent0=0):
    return "\n".join(
        indent(ln, ' '*(__indent0 if ln_i==0 else __indent))
        for ln_i, ln in enumerate(wrap(text, textwidth))
    )
def format_thread(thread):
    sio = StringIO()
    timestamp = datetime.strptime(t0.main.id[1:15], 
                                  "%Y%m%d%H%M%S").strftime("%c")
    sio.write(f"[{thread.source}] ")
    sio.write(timestamp + "\n")
    if thread.main:
        sio.write(format_text(thread.main.title, __indent=0))
        sio.write("\n")
        sio.write(format_text("----", __indent=0))
        sio.write("\n")
        sio.write(format_text(thread.main.text, __indent=0))
        sio.write("\n\n")
    for reply_x in thread.replies:
        sio.write(format_text(">> " + reply_x.text, __indent=3, __indent0=0))
        sio.write("\n")
    return sio.getvalue()

In [72]:
from io import StringIO
from textwrap import indent, wrap
from datetime import datetime

def format_thread_html(thread, idx=-1):
    sio = StringIO()
    op0 = thread.get_opinion()
    timestamp = datetime.strptime(op0.id[1:15], 
                                  "%Y%m%d%H%M%S").strftime("%c")
    sio.write(f"<div class='thread-wrapper' id='{op0.id}''>\n")
    sio.write(f"<span class='source'>{idx}. [{thread.source}] </span>")
    sio.write("<span class='timestamp'>" + timestamp + "</span>\n")
    if thread.main:
        sio.write("<div class='title'>")
        sio.write(format_text(thread.main.title, __indent=0))
        sio.write("</div>\n<div class='main-text'>\n")
        sio.write(format_text(thread.main.text, __indent=0))
        sio.write("</div>\n\n")
    else:
        sio.write("<div class='title'>(無主文)</div>")
    sio.write("<ol class='reply-wrap'>\n")
    for reply_x in thread.replies:
        sio.write(format_text("  <li class='reply-text'> " + reply_x.text, __indent=3, __indent0=0))
        sio.write("  </li>\n")
    sio.write("</ol>\n")
    sio.write("</div>")
    return sio.getvalue()

In [79]:
print(format_thread_html(threads[100]))

<div class='thread-wrapper' id='D2020010109440642954493560274790208575497087793916''>
<span class='source'>-1. [mobile01] </span><span class='timestamp'>Wed Jan  1 09:44:06 2020</span>
<div class='title'>(無主文)</div><ol class='reply-wrap'>
  <li class='reply-text'> 該不會是業者出來代風向球吧給你看看業者自己說的話喔5G標金破800億
   中華電謝繼茂：千億價格值得！請不要搗亂市場供需原則真懷疑你真是否消費哲  </li>
</ol>
</div>


In [74]:
len(threads)

56064

In [75]:
style_text = """
.wrapper {margin: auto; width: 50%; line-height: 150%}
.thread-wrapper {margin: 5% 0%; padding: 2% 2%; border-bottom: 1px solid gray}
.title {font-weight: bolder}
"""

In [80]:
fout = (casa.get_data_path() / "threads/threads-sample-every50.html").open("w", encoding="UTF-8")
fout.write("<!doctype html><html>\n")
fout.write("<head><style>")
fout.write(style_text)
fout.write("</style></head>\n")
fout.write("<body><div class='wrapper'>")
for counter, idx in enumerate(range(0, len(threads), 50)):
    try:
        html = format_thread_html(threads[idx], counter)
        fout.write(html)
    except Exception:
        pass
fout.write("</div></body>\n</html>")
fout.close()